# Inferencia completa

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 0) repositorio donde se encuentra almacenado el modelo
model_id = "Qwen/Qwen2.5-Coder-0.5B-Instruct"
# 1.1) Descarga tokenizer en tu ordenador y carga tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# 1.2) Descarga LLM en tu ordenador y lo carga en GPU y TPU
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.bfloat16
)

In [ ]:
# 2) Prompt en formato chat (instruct)
# el system prompt es un conjunto de instrucciones iniciales y 
# reglas de comportamiento configuradas en el LLM
# para definir su rol, personalidad, tono y restricciones antes de interactuar con el usuario.
messages = [
    {"role": "system", "content": "You are a helpful chatbot. Add comments to the code in a friendly tone."},
    {"role": "user", "content": "Write a Python function that sorts an" 
                                + " array using divide and conquer"} ]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(prompt)


In [ ]:
# 3) Tokenización
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# 4) Generación
outputs = model.generate(
    **inputs,
    max_new_tokens=1024,
    temperature=0.1,
    do_sample=True
)
# 5) Decodificación (solo del texto generado)
text = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[-1]:],
    skip_special_tokens=True
)
print(text)

# Inferencia con pipeline

In [ ]:
from transformers import pipeline
# 0) repositorio donde se encuentra almacenado el modelo
model_id = "Qwen/Qwen2.5-Coder-0.5B-Instruct"
# 1) Crear pipeline (descarga modelo + tokenizer y pone GPU/CPU automáticamente)
gen = pipeline(
    task="text-generation",
    model=model_id,
    device_map="auto"
)
# 2) Prompt en formato chat (instruct)
messages = [
    {"role": "system", "content": "You are a helpful chatbot. Add comments to the code in a friendly tone."},
    {"role": "user", "content": "Write a Python function that sorts an" 
                                + " array using divide and conquer"} ]

In [ ]:
# 3) Generación (pipeline devuelve texto ya decodificado)
out = gen(
    messages,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.1
)
print(out[0]["generated_text"][-1]["content"])